In [1]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
from tqdm import tqdm

In [2]:
df = pd.read_csv("cleaned_data/chats_clean.csv")

In [3]:
# preprocess function
def preprocess(text):
    new_text = []
    for t in str(text).split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
# load Model
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Exception in thread Thread-3:
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\_monitor.py", line 84, in run
    instance.refresh(nolock=True)
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\std.py", line 1347, in refresh
    self.display()
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\notebook.py", line 171, in display
    rtext.value = right
    ^^^^^^^^^^^
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\traitlets.py", line 716, in __set__
    self.set(obj, value)
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\traitlets.py", line 706, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "c:\Users\PC\AppData\Local\Programs\Pytho

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Exception ignored in: <function tqdm.__del__ at 0x000001EF27A35580>
Traceback (most recent call last):
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\PC\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm' object has no attribute 'disp'


In [6]:
# run sentiment analysis
sentiments = []
scores_list = []

for text in tqdm(df["question"].fillna("").tolist(), desc="Analyzing sentiments"):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512, padding=True)
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    label_id = np.argmax(scores)
    label = config.id2label[label_id]
    score = float(scores[label_id])

    sentiments.append(label)
    scores_list.append(score)
df["sentiment"] = sentiments
df["sentiment_score"] = scores_list

Analyzing sentiments: 100%|██████████| 11021/11021 [04:10<00:00, 43.92it/s]


In [ ]:
df.to_csv("cleaned_data/chats_sentiment.csv", index=False)

print("✅ Sentiment analysis completed successfully!")


✅ Sentiment analysis completed successfully!


KeyError: "['sentiment_label'] not in index"

In [10]:
print(df[["question", "sentiment", "sentiment_score"]].head())


                                          question sentiment  sentiment_score
0  do you recommend traveling to beirut right now?   neutral         0.944790
1          when is the next flight going to italy?   neutral         0.963298
2               hello, tell me more about yourself   neutral         0.807940
3                hello tell me more about yourself   neutral         0.783783
4                                who developed you   neutral         0.804216
